# 安装依赖

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
!pip install pysrt
from tqdm import tqdm
import pysrt
google_drive=False
import os

# 选择翻译方式

In [ ]:
#@title 翻译模型翻译方式，同时加载模型，查看可用语言（最后一步的源语言和目标语言只能从可用语言里面选，每个模型可用语言各不相同）
!pip install dl_translate
import dl_translate as dlt

翻译模型 = "facebook/nllb-200-distilled-600M" # @param ["facebook/m2m100_418M","facebook/m2m100_1.2B","facebook/mbart-large-50-many-to-many-mmt", "facebook/nllb-200-distilled-600M"]

mt = dlt.TranslationModel(翻译模型,device="auto")
print("可用语言：")
print("\n".join(mt.available_languages()))

In [ ]:
#@title 谷歌翻译方式（免费调用，不稳定，最后一步的源语言和目标语言只能从可用语言里面选）
!pip install googletrans==3.1.0a0
import googletrans
translator = googletrans.Translator()
print("可用语言：")
print("\n".join(googletrans.LANGUAGES))

In [ ]:
#@title ChatGLM3翻译方式（系统提示词中"{}"将被替换为目标语言）
系统提示词 = "\u4F60\u662F\u4E00\u4E2A\u8D1F\u8D23\u7FFB\u8BD1\u5B57\u5E55\u7684\u7A0B\u5E8F\u3002\u60A8\u7684\u4EFB\u52A1\u662F\u5C06\u8F93\u5165\u7684\u5B57\u5E55\u7684\u539F\u59CB\u8BED\u8A00\u7FFB\u8BD1\u4E3A{}\u3002DO NOT\u8F93\u51FA\u9664\u7FFB\u8BD1\u4EE5\u5916\u7684\u4EFB\u4F55\u6587\u672C\u3002" #@param {type: "string"}
!pip install transformers sentencepiece
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm3-6b", trust_remote_code=True)
model = AutoModel.from_pretrained("THUDM/chatglm3-6b", trust_remote_code=True, device='cuda')
model = model.eval()


# 连接到谷歌云盘
如果文件要保存到谷歌云盘，翻译步骤的“保存到”或“恢复文件”应以`drive/MyDrive/`开头

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 翻译
提示：如未指定恢复文件，则程序将不会保存恢复文件

In [ ]:
源文件 = '\u5F85\u7FFB\u8BD1.srt'  #@param {type: "string"}
保存到 = 'drive/MyDrive/\u7FFB\u8BD1.srt'  #@param {type: "string"}
源语言 = 'en'  #@param {type: "string"}
目标语言 = 'zh-cn'  #@param {type: "string"}
翻译方式 = "\u8C37\u6B4C\u7FFB\u8BD1" #@param ["谷歌翻译", "大模型翻译","ChatGLM3翻译","不翻译"]
恢复文件 = 'drive/MyDrive/\u6062\u590D\u6587\u4EF6.srt'  #@param {type: "string"}
恢复间隔 = 20  #@param {type: "integer"}
def translate(sentence):
    if 翻译方式=="大模型翻译":
        return mt.translate(sentence, source=源语言, target=目标语言)
    elif 翻译方式=="谷歌翻译":
        return translator.translate(sentence, src=源语言, dest=目标语言).text
    elif 翻译方式=="ChatGLM3翻译":
        response,now_history=model.chat(tokenizer,sentence,top_p=1,temperature=0.01,history=[{"role": "system", "content": 系统提示词.format(目标语言)}])
        return response
    elif 翻译方式=="不翻译":
        return sentence
def merge_subtitles(subtitles):
    merged_subtitles=pysrt.SubRipFile()
    current_sentence = ""
    current_start = None
    current_end = None
    for i in range(len(subtitles)):
        now_sub=subtitles[i]
        if i<len(subtitles)-1:
            next_sub=subtitles[i+1]
        #分割条件：当前句子结尾有“.”，下一个句子start与当前句子end不相等，已经是最后一个句子
        #若符合条件，则对当前句子进行分割
        if current_start is None:
            current_start=now_sub.start
        current_end=now_sub.end
        current_sentence+=now_sub.text+" "
        if i==len(subtitles)-1 or now_sub.text[-1]=="." or next_sub.start!=now_sub.end:
            #创建新的字幕项并添加到结果
            sub = pysrt.SubRipItem(index=len(merged_subtitles) + 1,
                                             start=current_start, end=current_end,
                                             text=current_sentence.strip())
            merged_subtitles.append(sub)
            #重置参数
            current_sentence = ""
            current_start = None
            current_end = None
    return merged_subtitles

def translate_with_progress(subtitles):
    print("对字幕进行合并中...")
    merged_subtitles=merge_subtitles(subtitles)
    print("字幕合并完成，开始翻译")
    # 是否进行恢复
    if 恢复文件:
        #如果输入了恢复文件
        if not os.path.exists(恢复文件):
            #如果恢复文件不存在，则创建它
            print("恢复文件不存在，已创建新的恢复文件")
            with open(恢复文件, "w") as f:pass
            recover_index = 0
            translated_subs = pysrt.SubRipFile()
        else:
            # 读取恢复文件
            recovered_subs = pysrt.open(恢复文件, encoding='utf-8')
            #有恢复文件，从恢复文件结尾开始继续
            recover_index=len(recovered_subs)
            translated_subs = recovered_subs
            print("找到恢复文件，从第{}句继续翻译".format(recover_index))
    else:
        recover_index = 0
        translated_subs = pysrt.SubRipFile()

    # 使用 tqdm 显示进度条
    progress_bar = tqdm(total=len(merged_subtitles), desc="翻译中", unit="个")
    progress_bar.update(recover_index)
    for i in range(recover_index, len(merged_subtitles)):
        translated = pysrt.SubRipItem(index=len(translated_subs) + 1,
                                             start=merged_subtitles[i].start, end=merged_subtitles[i].end,
                                             text=translate(merged_subtitles[i].text).strip())
        translated_subs.append(translated)
        progress_bar.update(1)
        if 恢复文件 and i%恢复间隔==0:
            translated_subs.save(恢复文件,encoding='utf-8')
    # 关闭进度条
    progress_bar.close()
    return translated_subs

# 读取SRT文件
subs = pysrt.open(源文件, encoding='utf-8')

# 获取翻译后的字幕
translated_subs = translate_with_progress(subs)
print("\n-------------------------\n翻译完毕！")
# 保存翻译后的SRT文件
translated_subs.save(保存到, encoding='utf-8')
print("文件保存到了：{}".format(保存到))